In [1]:
import pandas as pd

In [2]:

#NLTK
import nltk
import nltk.corpus 
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /home/surya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:

import DataPrep
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import  LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, f1_score, classification_report

[nltk_data] Downloading package stopwords to /home/surya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
train = pd.read_csv('./dataset/normal/train.csv')
test = pd.read_csv('./dataset/normal/test.csv')
val = pd.read_csv('./dataset/normal/val.csv')

In [5]:
test = pd.concat([test,val], ignore_index= True)
len(test)

2538

In [6]:
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [7]:
# Bag of words
bow_transformer = CountVectorizer(analyzer= text_process).fit(train['statement'])
print(len(bow_transformer.vocabulary_))

14652


In [8]:
# Naive Baye's

NaiveB = Pipeline([
                   ('bow', CountVectorizer(analyzer=text_process, lowercase=True)),
                   ('classifier', MultinomialNB()),
])

In [9]:
NaiveB.fit(train['statement'], train['label'])

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function text_process at 0x7fb25b8e33a0>)),
                ('classifier', MultinomialNB())])

In [10]:
NaiveB_predicted = NaiveB.predict(test['statement'])
print(classification_report(test['label'], NaiveB_predicted))
print('\n')
print(confusion_matrix(test['label'], NaiveB_predicted))

              precision    recall  f1-score   support

           0       0.59      0.48      0.53      1163
           1       0.62      0.72      0.67      1375

    accuracy                           0.61      2538
   macro avg       0.61      0.60      0.60      2538
weighted avg       0.61      0.61      0.60      2538



[[554 609]
 [382 993]]


In [11]:
#logistic Regression
logR = Pipeline([
                 ('bow', CountVectorizer(analyzer=text_process, lowercase=True)),
                 ('logR',LogisticRegression())
])

In [12]:
logR.fit(train['statement'], train['label'])

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function text_process at 0x7fb25b8e33a0>)),
                ('logR', LogisticRegression())])

In [13]:
logR_predicted = logR.predict(test['statement'])
print(classification_report(test['label'], logR_predicted))
print('\n')
print(confusion_matrix(test['label'], logR_predicted)) 

              precision    recall  f1-score   support

           0       0.57      0.51      0.54      1163
           1       0.62      0.67      0.64      1375

    accuracy                           0.60      2538
   macro avg       0.59      0.59      0.59      2538
weighted avg       0.59      0.60      0.59      2538



[[593 570]
 [452 923]]


In [14]:
#Random forest

randomForest = Pipeline([
                         ('bow', CountVectorizer(analyzer=text_process, lowercase=True)),
                         ('randForestCls',RandomForestClassifier(n_estimators=200))
])

In [15]:
randomForest.fit(train['statement'], train['label'])

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function text_process at 0x7fb25b8e33a0>)),
                ('randForestCls', RandomForestClassifier(n_estimators=200))])

In [16]:
randomForest_predicted = randomForest.predict(test['statement'])
print(classification_report(test['label'], randomForest_predicted))
print('\n')
print(confusion_matrix(test['label'], randomForest_predicted)) 

              precision    recall  f1-score   support

           0       0.61      0.45      0.52      1163
           1       0.62      0.76      0.68      1375

    accuracy                           0.62      2538
   macro avg       0.61      0.60      0.60      2538
weighted avg       0.61      0.62      0.61      2538



[[ 518  645]
 [ 330 1045]]


In [17]:
#svm classifier
svm_ = Pipeline([
                ('bow', CountVectorizer(analyzer=text_process, lowercase=True)),
                ('svm_clsf', svm.LinearSVC()),
                
])

In [18]:
svm_.fit(train['statement'], train['label'])

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function text_process at 0x7fb25b8e33a0>)),
                ('svm_clsf', LinearSVC())])

In [19]:
svm_predicted = svm_.predict(test['statement'])
print(classification_report(test['label'], svm_predicted))
print('\n')
print(confusion_matrix(test['label'], svm_predicted)) 


              precision    recall  f1-score   support

           0       0.53      0.52      0.53      1163
           1       0.60      0.62      0.61      1375

    accuracy                           0.57      2538
   macro avg       0.57      0.57      0.57      2538
weighted avg       0.57      0.57      0.57      2538



[[605 558]
 [528 847]]


In [20]:
sgd = Pipeline([
                ('bow', CountVectorizer(analyzer=text_process, lowercase=True)),
                ('sgd', SGDClassifier())
])

In [21]:
sgd.fit(train['statement'], train['label'])

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function text_process at 0x7fb25b8e33a0>)),
                ('sgd', SGDClassifier())])

In [22]:
sgd_predicted = sgd.predict(test['statement'])
print(classification_report(test['label'], sgd_predicted))
print('\n')
print(confusion_matrix(test['label'], sgd_predicted)) 


              precision    recall  f1-score   support

           0       0.55      0.52      0.53      1163
           1       0.61      0.64      0.63      1375

    accuracy                           0.58      2538
   macro avg       0.58      0.58      0.58      2538
weighted avg       0.58      0.58      0.58      2538



[[601 562]
 [493 882]]
